In [23]:
import numpy as np
import pandas as pd
from collections import Counter

In [24]:
#Mengimport data testing
dftest = pd.read_csv('C:/Users/SMK Telkom Malang/Documents/TEAM 2 AI/data_test.csv', delimiter=';')
dftest.drop('No. Urut', axis=1, inplace=True)
dftest.dropna(inplace=True) ##

#Mengimport data training
dftrain = pd.read_csv('C:/Users/SMK Telkom Malang/Documents/TEAM 2 AI/data_train.csv', delimiter=';')
dftrain.drop('No', axis=1, inplace=True)

#Mengatasi data Null di kolom Huruf
dftrain['HURUF'].fillna(value='NA', inplace=True)

#Mengelompokkan kolom zone
cols = dftrain.columns[1:]

# Metode Z score
dfz = dftrain.copy()
for i in cols:
    dfz[i] = (dftrain[i]-dftrain[i].mean())/dftrain[i].std()

#print(dfz.head())

#Menggunakan metode Minmaxscaler
dfmms = dftrain.copy()
for a in cols:
    dfmms[a] = (dftrain[a] - dftrain[a].min()) / (dftrain[a].max() - dftrain[a].min())
# print(dfmms.head())

#Mempersiapkan data untuk training
x = dfz.drop('HURUF', axis=1)
y = dfz.HURUF

In [25]:
x

,Z1,Z2,Z3,Z4,Z5,Z6,Z7,Z8,Z9,Z10,Z11,Z12,Z13,Z14,Z15,Z16
0,1.184164,-0.483281,0.599098,-0.378007,0.503118,0.680363,-0.026111,-0.190653,-0.134226,0.852160,-0.260252,-0.241015,0.166938,0.978193,-0.078989,0.411856
1,-0.595716,-1.220924,-0.811371,-1.339107,-0.920629,-0.975616,-0.038175,-0.842401,-0.873171,-1.056279,0.349132,-1.322495,-0.939210,-0.662214,-0.125226,-1.213735
2,1.239785,0.692672,1.824748,0.965217,2.720878,0.821571,0.758035,0.446283,2.684712,0.208485,0.745232,0.500570,3.293008,1.634357,0.741723,1.348338
3,-0.840449,-0.686400,0.404551,-0.991721,-0.770040,-0.654689,0.552951,-1.005338,-0.736330,-0.717503,0.166317,-1.059850,-0.618588,-0.115412,-0.356412,-0.701321
4,1.250909,-0.878829,0.920102,-0.134837,0.434669,0.333763,1.337097,0.179659,0.153141,1.111888,0.785858,0.207026,0.134875,1.218787,-0.044311,0.588550
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445,-0.517846,-0.333614,-1.005919,-0.262212,-0.660521,-0.654689,-1.292809,-1.049775,-0.490015,-0.672333,-1.316519,-1.075300,-0.602557,-1.099657,-0.229260,-0.277254
446,-0.395479,0.414719,0.122457,0.502036,0.106112,-0.025674,-0.219132,-0.338777,-0.065806,-0.209339,-1.316519,-0.426412,-0.778899,-0.126348,0.117520,0.111475
447,0.060615,-1.252996,0.229458,0.386241,0.366219,-0.269578,-1.292809,-0.694276,-0.134226,-0.073828,-1.296207,-0.519110,-0.554463,-0.476302,-0.865022,0.040797
448,-1.285419,-0.130495,1.737202,-0.505382,-0.304585,-0.141208,-0.556918,-0.398027,-0.257384,-0.243216,-1.316519,-0.333714,0.022657,1.546868,-0.402649,0.235161


In [26]:
y

0       HA
1       HA
2       HA
3       HA
4       HA
      ... 
445    NGA
446    NGA
447    NGA
448    NGA
449    NGA
Name: HURUF, Length: 450, dtype: object

.

.

# Data Training

## Algoritma KNN

1. Menghitung jarak ke semua Titik menggunakan Euclidean Distance
2. Mengambil titik terdekat sebanyak K
3. Mengambil label class terbaik
4. Melakukan voting terhadap label terbanyak

In [27]:
from collections import Counter

class KNN:
    def __init__(self, k=3):
        self.K = k
        self.k_terbaik = []

    def train(self, xtrain, ytrain):
        if isinstance(xtrain, pd.DataFrame):
            xtrain = xtrain.values

        self.x_train = xtrain
        self.y_train = ytrain
    
    def predict(self, xpred):
        if isinstance(xpred, pd.DataFrame):
            xpred = xpred.values

        #Melakukan prediksi label untuk setiap nilai x yang dimasukkan ke dalam xpred
        y_pred = [self._prediksi(x) for x in xpred]
        return np.array(y_pred)

    def _prediksi(self, x):
        #1. Menghitung jarak ke semua titik
        jarak_titik = [self.hitungjarak(x, x_piece) for x_piece in self.x_train]

        #2. Mengambil Titik terdekat sebanyak "K"
        k_terbaik = np.argsort(jarak_titik)[:self.K]
        # print(k_terbaik)

        #3. Mengambil label class dari k_terbaik
        label_k_terbaik = [self.y_train[i] for i in k_terbaik]
        self.k_terbaik.append(label_k_terbaik)
    
        #4. Melakukan voting label mayoritas
        hasil_voting = Counter(label_k_terbaik).most_common(1)
        # print(hasil_voting)
        
        return hasil_voting[0][0]

    def hitungjarak(self, x1, x2):
        #Euclidean distance
        jarak = np.sqrt(np.sum((x1 - x2)**2))
        return jarak
    
    
    def score(self, xtest, ytest):
        prediction = self.predict(xtest)
        akurasi = np.sum(prediction == ytest) / len(x_test)
        return akurasi

### Algoritma Test Split

In [28]:
def TestSplit(x, y, testsize=0.75):
    x_shuffle = x.sample(frac=1)
    
    testsize = int(testsize * len(x))
    
    x_test = x_shuffle[testsize:]
    y_test = [y[i] for i in x_test.index]
    
    return x_test, y_test

.

In [29]:
x_test, y_test = TestSplit(x, y)

In [30]:
print(x.shape)
print(x_test.shape)

(450, 16)
(113, 16)


Contoh hasil prediksi

In [31]:
model = KNN(k=3)
model.train(x, y)
model.predict(x_test)

array(['YA', 'PA', 'WA', 'YA', 'SA', 'TA', 'TA', 'DHA', 'PA', 'SA', 'LA',
       'PA', 'MA', 'TA', 'LA', 'NA', 'HA', 'NA', 'HA', 'RA', 'NYA', 'NA',
       'BA', 'HA', 'KA', 'WA', 'CA', 'NYA', 'NA', 'CA', 'DA', 'RA', 'DHA',
       'KA', 'PA', 'JA', 'KA', 'CA', 'KA', 'SA', 'DHA', 'YA', 'MA', 'RA',
       'MA', 'SA', 'DA', 'THA', 'DA', 'HA', 'JA', 'NGA', 'TA', 'KA', 'RA',
       'NGA', 'NGA', 'RA', 'BA', 'NA', 'NGA', 'KA', 'NYA', 'BA', 'SA',
       'TA', 'RA', 'BA', 'JA', 'THA', 'HA', 'GA', 'BA', 'DA', 'RA', 'THA',
       'TA', 'RA', 'WA', 'NGA', 'GA', 'BA', 'RA', 'NA', 'DA', 'DA', 'HA',
       'MA', 'NA', 'PA', 'CA', 'MA', 'HA', 'DA', 'BA', 'WA', 'NGA', 'NYA',
       'NA', 'GA', 'SA', 'RA', 'NGA', 'HA', 'KA', 'DA', 'THA', 'NA', 'KA',
       'MA', 'DHA', 'JA', 'TA'], dtype='<U3')

In [32]:
model.k_terbaik

[['YA', 'PA', 'YA'],
 ['PA', 'PA', 'PA'],
 ['WA', 'JA', 'WA'],
 ['YA', 'HA', 'WA'],
 ['SA', 'SA', 'PA'],
 ['TA', 'TA', 'BA'],
 ['TA', 'DHA', 'WA'],
 ['DHA', 'CA', 'WA'],
 ['PA', 'PA', 'PA'],
 ['SA', 'CA', 'HA'],
 ['LA', 'JA', 'YA'],
 ['PA', 'PA', 'YA'],
 ['MA', 'MA', 'LA'],
 ['TA', 'HA', 'DHA'],
 ['LA', 'YA', 'LA'],
 ['NA', 'LA', 'PA'],
 ['HA', 'PA', 'WA'],
 ['NA', 'TA', 'DA'],
 ['HA', 'WA', 'YA'],
 ['RA', 'RA', 'RA'],
 ['NYA', 'TA', 'WA'],
 ['NA', 'NA', 'HA'],
 ['BA', 'BA', 'WA'],
 ['HA', 'BA', 'CA'],
 ['KA', 'GA', 'NA'],
 ['WA', 'WA', 'DHA'],
 ['CA', 'CA', 'CA'],
 ['BA', 'NYA', 'NYA'],
 ['NA', 'NA', 'PA'],
 ['CA', 'HA', 'WA'],
 ['DA', 'DA', 'DA'],
 ['RA', 'RA', 'RA'],
 ['DHA', 'DHA', 'DHA'],
 ['KA', 'KA', 'NA'],
 ['PA', 'PA', 'WA'],
 ['JA', 'JA', 'HA'],
 ['KA', 'NA', 'KA'],
 ['CA', 'NA', 'KA'],
 ['KA', 'HA', 'YA'],
 ['SA', 'DA', 'PA'],
 ['DHA', 'PA', 'WA'],
 ['JA', 'YA', 'YA'],
 ['MA', 'PA', 'MA'],
 ['RA', 'RA', 'RA'],
 ['MA', 'WA', 'YA'],
 ['SA', 'BA', 'PA'],
 ['DA', 'JA', 'DA'],
 [

In [33]:
Kterbaik = pd.DataFrame(model.k_terbaik, columns=['k1','k2','k3'])
Kterbaik['hasil Voting'] = model.predict(x_test)

In [34]:
Kterbaik.head(20)

,k1,k2,k3,hasil Voting
0,YA,PA,YA,YA
1,PA,PA,PA,PA
2,WA,JA,WA,WA
3,YA,HA,WA,YA
4,SA,SA,PA,SA
5,TA,TA,BA,TA
6,TA,DHA,WA,TA
7,DHA,CA,WA,DHA
8,PA,PA,PA,PA
9,SA,CA,HA,SA


In [39]:
model = KNN(k=7)
model.train(x, y)
model.score(x_test, y_test)

0.7079646017699115

### Mencoba Training menggunakan data hasil MinMax

In [ ]:
x = dfmms.drop('HURUF', axis=1)
y = dfmms['HURUF']

x_test, y_test = TestSplit(x, y)

for k in range(3, 30):
    model = KNN(k)
    model.train(x, y)
    akurasi = model.score(x_test, y_test)
    if akurasi > 0.9:
        print(f'K = {k}, Akurasi = {akurasi}')
    else:
        continue

.

### Mencoba Training menggunaan data hasil ZScore

In [41]:
x = dfz.drop('HURUF', axis=1)
y = dfz['HURUF']

x_test, y_test = TestSplit(x, y)

for k in range(3, 30):
    model = KNN(k)
    model.train(x, y)
    akurasi = model.score(x_test, y_test)
    if akurasi > 0.9:
        print(f'K = {k}, Akurasi = {akurasi}')
    else:
        continue

K = 3, Akurasi = 0.9026548672566371


In [72]:


model = KNN(k=3)
model.train(x, y)
model.score(x_test, y_test)

0.9469026548672567

.

## Percobaan eksekusi menggunakan data test asli

In [ ]:
x_train = dfz.drop('HURUF', axis=1)
y_train = dfz['HURUF']

Data Preprocessing untuk dftest

In [ ]:
dfz_test = dftest.copy()
for i in cols:
    dfz_test[i] = (dftest[i]-dftest[i].mean())/dftest[i].std()

In [ ]:
x_test = dfz_test.values

In [ ]:
x_test

In [ ]:
model = KNN(k=3)
model.train(x, y)
model.predict(x_test)

In [ ]:
model.predict(dftest[12:13])[0]

In [ ]:
in =

In [ ]:
in = np.array(in)
in_zscore = (in - np.mean(in)) / np.std(in)

x = dfz.drop('HURUF', axis=1)
y = dfz['HURUF']

model = KNN(k=3)
model.train(x, y)
model.predict(in_zscore)